<a href="https://colab.research.google.com/github/elangbijak4/Blockchain_Research/blob/main/Blockchain_PoBFT_with_Reward.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import hashlib
import datetime
import json
import random

In [2]:
class Block:
    def __init__(self, index, previous_hash, timestamp, data, validator, reward):
        self.index = index
        self.previous_hash = previous_hash
        self.timestamp = timestamp
        self.data = data
        self.validator = validator
        self.reward = reward
        self.hash = self.calculate_hash()

    def calculate_hash(self):
        block_string = f"{self.index}{self.previous_hash}{self.timestamp}{self.data}{self.validator}{self.reward}"
        return hashlib.sha256(block_string.encode()).hexdigest()

    def to_dict(self):
        return {
            'index': self.index,
            'previous_hash': self.previous_hash,
            'timestamp': self.timestamp,
            'data': self.data,
            'validator': self.validator,
            'reward': self.reward,
            'hash': self.hash
        }

In [3]:
class Blockchain:
    def __init__(self, validators, bft, filename='blockchain.json'):
        self.validators = validators
        self.bft = bft
        self.filename = filename
        self.rewards = {v: 0 for v in validators}  # Initialize rewards for each validator
        try:
            with open(self.filename, 'r') as f:
                self.chain = self.load_from_json(f.read())
        except FileNotFoundError:
            self.chain = [self.create_genesis_block()]
            self.save_to_json()

    def create_genesis_block(self):
        return Block(0, "0", self.get_utc_time(), "Genesis Block", self.validators[0], 0)

    def get_latest_block(self):
        return self.chain[-1]

    def add_block(self, data):
        previous_block = self.get_latest_block()
        validator = self.bft.select_validator(len(self.chain))
        reward = self.bft.calculate_reward(validator)
        new_block = Block(len(self.chain), previous_block.hash, self.get_utc_time(), data, validator, reward)
        if self.bft.reach_consensus(new_block, self.chain):
            self.chain.append(new_block)
            self.rewards[validator] += reward  # Update the reward for the validator
            self.save_to_json()

    def is_chain_valid(self):
        for i in range(1, len(self.chain)):
            current_block = self.chain[i]
            previous_block = self.chain[i - 1]

            if current_block.hash != current_block.calculate_hash():
                return False
            if current_block.previous_hash != previous_block.hash:
                return False
        return True

    def get_utc_time(self):
        return datetime.datetime.utcnow().timestamp()

    def display_chain(self):
        with open(self.filename, 'r') as f:
            blockchain_data = json.load(f)
            for block_data in blockchain_data:
                block_data.pop('hash', None)
                block = Block(**block_data)
                print(f"Block {block.index}:")
                print(f"  Timestamp: {datetime.datetime.utcfromtimestamp(block.timestamp)}")
                print(f"  Data: {block.data}")
                print(f"  Validator: {block.validator}")
                print(f"  Reward: {block.reward}")
                print(f"  Hash: {block.hash}")
                print(f"  Previous Hash: {block.previous_hash}\n")

    def save_to_json(self):
        with open(self.filename, 'w') as f:
            json.dump([block.to_dict() for block in self.chain], f, indent=4)

    def load_from_json(self, json_string):
        data = json.loads(json_string)
        blocks = []
        for block_data in data:
            block_data.pop('hash', None)
            block = Block(**block_data)
            blocks.append(block)
        return blocks

    def display_rewards(self):
        for validator, reward in self.rewards.items():
            print(f"Validator {validator} has a reward of {reward}")

In [4]:
class ByzantineFaultTolerance:
    def __init__(self, validators, faulty_validators=None):
        self.validators = validators
        self.faulty_validators = faulty_validators if faulty_validators else []
        self.leader = None

    def select_leader(self):
        non_faulty_validators = [v for v in self.validators if v not in self.faulty_validators]
        self.leader = random.choice(non_faulty_validators)
        return self.leader

    def select_validator(self, chain_length):
        non_faulty_validators = [v for v in self.validators if v not in self.faulty_validators]
        return random.choice(non_faulty_validators)

    def validate_block(self, block, chain):
        previous_block = chain[block.index - 1]
        if block.previous_hash != previous_block.hash:
            return False
        if block.hash != block.calculate_hash():
            return False
        return True

    def add_faulty_validator(self, validator):
        if validator not in self.faulty_validators:
            self.faulty_validators.append(validator)

    def remove_faulty_validator(self, validator):
        if validator in self.faulty_validators:
            self.faulty_validators.remove(validator)

    def reach_consensus(self, block, chain):
        votes = {validator: self.validate_block(block, chain) for validator in self.validators if validator not in self.faulty_validators}
        positive_votes = sum(votes.values())
        if positive_votes > len(self.validators) * 2 / 3:
            return True
        return False

    def calculate_reward(self, validator):
        # For simplicity, we set a fixed reward. In practice, it might depend on various factors.
        return 10

Demo BFT

In [5]:
# Implementasi BFT dengan kemampuan mengubah status validator dan memberikan reward
validators = ["Adil", "Rimba", "Nita"]
bft = ByzantineFaultTolerance(validators)

In [6]:
# Pemilihan leader untuk siklus konsensus
leader = bft.select_leader()
print(f"Leader yang dipilih: {leader}")

Leader yang dipilih: Rimba


In [7]:
# Menandai "Nita" sebagai validator bermasalah
bft.add_faulty_validator("Nita")

blockchain = Blockchain(validators, bft)

In [8]:
# Menambahkan blok baru
blockchain.add_block("Data untuk blok 1")
blockchain.add_block("Data untuk blok 2")

In [9]:
# Mengubah status "Nita" menjadi jujur
bft.remove_faulty_validator("Nita")

In [10]:
# Menambahkan blok lagi
blockchain.add_block("Data untuk blok 3")

blockchain.display_chain()
blockchain.display_rewards()

Block 0:
  Timestamp: 2024-06-27 06:01:09.777925
  Data: Genesis Block
  Validator: Adil
  Reward: 0
  Hash: fc4afa68c73785338bddcf849b0fcfabd749f4514d2410e53e9216195c4a9971
  Previous Hash: 0

Block 1:
  Timestamp: 2024-06-27 06:01:31.516865
  Data: Data untuk blok 3
  Validator: Rimba
  Reward: 10
  Hash: c14be96ba1c67c00411cf825a7de491f2a56113c1b65de23004519e02b6d4739
  Previous Hash: fc4afa68c73785338bddcf849b0fcfabd749f4514d2410e53e9216195c4a9971

Validator Adil has a reward of 0
Validator Rimba has a reward of 10
Validator Nita has a reward of 0
